In [ ]:
from station_data import *
from stream_processing import *
from event_detection import *
from catalogued_events import *
from stream_processing import *
import matplotlib.pyplot as plt
from obspy import UTCDateTime
import torch
import sys
from report_generation import *
import matplotlib.pyplot as plt
from obspy import UTCDateTime
import torch
import sys


In [4]:
denoiser_model = sbm.DeepDenoiser.from_pretrained("original")
phase_picking_model = sbm.EQTransformer.from_pretrained("original")
station = ['AM','R50D6','https://data.raspberryshake.org']
station_coordinates = get_coordinates(station)
providers = ["IRIS", "USGS", "EMSC"]

# Initialize DataFrame to store results
columns = ['date', 'number_catalogued', 'number_matched', 'number_not_detected', 'number_not_in_catalogue', 'number_both_phases_identified']
results_df = pd.DataFrame(columns=columns)

# Define the date range
start_date = datetime.date(2024, 2, 8)
end_date = datetime.date.today()

# Iterate over each day
current_date = start_date

In [5]:

# Define the path for saving the CSV file
output_file = 'earthquake_stats.csv'  # Ensure this points to the correct path where you want the file saved

while current_date <= end_date:
    date = UTCDateTime(current_date)
    date_folder = generate_data_path(date, station)
    
    try:
        stream_file = download_stream(date, station, path=date_folder, overwrite=False)
        if not stream_file:
            raise FileNotFoundError("Stream file not downloaded.")

        original_stream = read(stream_file)
        processed_stream = stream_process(original_stream)
        denoised_stream = deep_denoiser(processed_stream, denoiser_model)
        
        df_phase_picked, annotated_stream = predict_and_annotate(denoised_stream, phase_picking_model)
        df_phase_picked = df_phase_picked[df_phase_picked['peak_confidence'] >= 0.7]

        catalogue_path = request_catalogue(
            catalogue_providers=providers,
            station_information=station,
            station_location=station_coordinates,
            date=date,
            radmin=0,
            radmax=90,
            minmag=3,
            maxmag=10,
            path=date_folder,
            overwrite=True
        )

        if not catalogue_path:
            raise FileNotFoundError("Catalogue file not downloaded.")

        catalogue = load_earthquake_catalog(catalogue_path)
        df_catalogued = process_catalogue(catalogue, station_coordinates)
        result_df = match_and_merge(df_catalogued, df_phase_picked, time_tolerance=10)
        stats = calculate_matching_stats(result_df)

        # Create a new DataFrame for the current day's results
        day_results = pd.DataFrame({
            'date': [current_date.strftime('%Y-%m-%d')],
            'number_catalogued': [stats[0]],
            'number_matched': [stats[1]],
            'number_not_detected': [stats[2]],
            'number_not_in_catalogue': [stats[3]],
            'number_both_phases_identified': [stats[4]]
        })
        results_df = pd.concat([results_df, day_results], ignore_index=True)
        
        # Save results to CSV after each successful loop iteration
        results_df.to_csv(output_file, mode='a', header=False, index=False)

    except FileNotFoundError as e:
        print(f"Skipping {current_date.strftime('%Y-%m-%d')}: {str(e)}")

    # Move to the next day
    print("Day processed: ", current_date.strftime('%Y-%m-%d'))
    current_date += datetime.timedelta(days=1)

print("Completed. Results saved to 'earthquake_stats.csv'.")


Data for 2024-02-08 already exist.
CUDA available. Running on GPU
CUDA available. Running on GPU


KeyboardInterrupt: 